**Jose Dixon**<br>
**COSC 611**<br>
**Final Project- Text Mining Analysis of COVID19 Papers**<br>
**December 3, 2021**<br>
**Disclaimer: I was able to run code successfully in Google Colab. It was more difficult to run with PySpark and Jupyter Notebook with my computer.**



In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://apache.osuosl.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop2.7.tgz

In [3]:
!tar xf spark-3.1.3-bin-hadoop2.7.tgz

In [4]:
pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop2.7"

In [6]:
os.environ["SPARK_HOME"]

'/content/spark-3.1.2-bin-hadoop2.7'

In [7]:
pip install spark-nlp==2.4.2

     |████████████████████████████████| 108 kB 5.4 MB/s 


In [8]:
pip install pyspark==3.1.3

     |████████████████████████████████| 212.4 MB 67 kB/s 
     |████████████████████████████████| 198 kB 58.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=145431fcb0a9a7a6050a212f87c5530f54fc6762ded179a8404f7d067c728280
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [9]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.4.2
Apache Spark version:  3.1.2


In [10]:
pip install langid

     |████████████████████████████████| 1.9 MB 5.3 MB/s 
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941187 sha256=bf390963266c3e3aba3f344e544272f89e0159e85c11768224f50920bec51d83
  Stored in directory: /root/.cache/pip/wheels/2b/bb/7f/11e4db39477278161e882eadc46fb558949a28b13470fc74b8
Successfully built langid


In [11]:
pip install nltk

In [12]:
import string

In [2]:
# 14.2. Text Preprocessing
# NOTE: Rerun from here if you get SparkContexts error. 
def check_blanks(data_str):
    is_blank = str(data_str.isspace())
    return is_blank

In [ ]:
'''
Ignore this
import langid
def check_lang(data_str):
    predict_lang = langid.classify(data_str)
    if predict_lang[1] >= .9:
        language = predict_lang[0]
    else:
        language = 'NA'
    return language
'''

"\nimport langid\ndef check_lang(data_str):\n    predict_lang = langid.classify(data_str)\n    if predict_lang[1] >= .9:\n        language = predict_lang[0]\n    else:\n        language = 'NA'\n    return language\n"

In [3]:
# Remove features
import re
import string
def remove_features(data_str):
    # compile regex
    url_re = re.compile('https?://(www.)?\w+\.\w+(/\w+)*/?')
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    punc_re = re.compile('[^A-Za-z0-9]+') 
    num_re = re.compile('(\\d+)')
    mention_re = re.compile('@(\w+)')
    alpha_num_re = re.compile("^[a-z0-9_.]+$")
    # convert to lowercase
    data_str = data_str.lower()
    # remove hyperlinks
    data_str = url_re.sub(' ', data_str)
    # remove @mentions
    data_str = mention_re.sub(' ', data_str)
    # remove puncuation
    data_str = punc_re.sub(' ', data_str)
    # remove numeric 'words'
    data_str = num_re.sub(' ', data_str)
    # remove non a-z 0-9 characters and words shorter than 3 characters
    list_pos = 0
    cleaned_str = ''
    for word in data_str.split():
        if list_pos == 0:
            if alpha_num_re.match(word) and len(word) > 2:
                cleaned_str = word
            else:
                cleaned_str = ' '
        else:
            if alpha_num_re.match(word) and len(word) > 2:
                cleaned_str = cleaned_str + ' ' + word
            else:
                cleaned_str += ' '
        list_pos += 1
    return cleaned_str

In [4]:
# Removes stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords.words('english')
print(stopwords.words('english'))
#print(stops)
def remove_stops(data_str):
    # expects a string
    stopwords = nltk.corpus.stopwords.words('english')
    list_pos = 0
    cleaned_str = ''
    text = data_str.split()
    for word in text:
        if word not in stopwords:
            # rebuild cleaned_str
            if list_pos == 0:
                cleaned_str = word
            else:
                cleaned_str = cleaned_str + ' ' + word
            list_pos += 1
    return cleaned_str

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'bo

In [5]:
# Tagging text
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
def tag_and_remove(data_str):
    cleaned_str = ' '
    # noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
    # adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
    # verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    nltk_tags = nn_tags + jj_tags + vb_tags

    # break string into 'words'
    text = data_str.split()

    # tag the text and keep only those with the right tags
    tagged_text = nltk.pos_tag(text)
    for tagged_word in tagged_text:
        if tagged_word[1] in nltk_tags:
            cleaned_str += tagged_word[0] + ' '

    return cleaned_str

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [6]:
# Lemmatization 
import re
import string
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
def lemmatize(data_str):
    # expects a string
    list_pos = 0
    cleaned_str = ''
    lmtzr = WordNetLemmatizer()
    text = data_str.split()
    tagged_words = nltk.pos_tag(text)
    for word in tagged_words:
        if 'v' in word[1].lower():
            lemma = lmtzr.lemmatize(word[0], pos='v')
        else:
            lemma = lmtzr.lemmatize(word[0], pos='n')
        if list_pos == 0:
            cleaned_str = lemma
        else:
            cleaned_str = cleaned_str + ' ' + lemma
        list_pos += 1
    return cleaned_str

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
'''
# Functions provided for preprocessing - Please ignore
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
#import preproc as pp
check_lang_udf = udf(check_lang, StringType())
remove_stops_udf = udf(remove_stops, StringType())
remove_features_udf = udf(remove_features, StringType())
tag_and_remove_udf = udf(tag_and_remove, StringType())
lemmatize_udf = udf(lemmatize, StringType())
check_blanks_udf = udf(check_blanks, StringType())
'''

NameError: ignored

In [7]:
# Execution for loading dataset
import pyspark
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

#import preproc as pp
'''
check_lang_udf = udf(check_lang, StringType())
'''
remove_stops_udf = udf(remove_stops, StringType())
remove_features_udf = udf(remove_features, StringType())
tag_and_remove_udf = udf(tag_and_remove, StringType())
lemmatize_udf = udf(lemmatize, StringType())
check_blanks_udf = udf(check_blanks, StringType())

# create Spark context with Spark configuration

sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)
'''
# import
from pyspark.sql import HiveContext
sc = pyspark.SparkContext()
sqlContext = HiveContext(sc)
#spark = SparkSession.builder.appName("DataFrame").getOrCreate()
'''
# Uncomment here
# Read file into RDD

test_rdd = sc.textFile("testing.txt")
#prob_rdd = sc.textFile("problems_in_china.txt")
#imm_rdd = sc.textFile("immune.txt")
#trans_rdd = sc.textFile("transmission.txt")
#risk_rdd = sc.textFile("riskfactors.txt")

#parts_rdd = data_rdd.map(lambda l: l.split(","))
# Each line is converted to a tuple.
#tuple_rdd = parts_rdd.map(lambda p: Row(id=int(p[0]), order_date=p[1], customer_id=p[2], order_status=p[3]))
#filled_rdd = tuple_rdd.filter(bool)


#lines.toDF()
#lines = spark.read.text("immune.txt")
# Call collect() to get all data
#llist = lines.collect()

# print line one by line
#for line in llist:
#	print(line)
# import
'''
# Then, you can use the com.databricks.spark.csv.
lines = sqlContext.read \
     .format('com.databricks.spark.csv') \
     .options(header='false', delimiter='|') \
     .load('immune.txt')
'''
parts_rdd = test_rdd.map(lambda l: l.split("\t"))
# Filter bad rows out
filled_rdd = parts_rdd.filter(bool)
#typed_rdd = data_rdd.map(lambda p: (p[0], p[1], float(p[2])))

#Create DataFrame
data_df = sqlContext.createDataFrame(filled_rdd, StringType())
#data_df = sqlContext.createDataFrame(prob_rdd, StringType())
#data_df = sqlContext.createDataFrame(imm_rdd, StringType())
#data_df = sqlContext.createDataFrame(trans_rdd, StringType())
#data_df = sqlContext.createDataFrame(risk_rdd, StringType())

# Ignore
#label_df = data_df.withColumn('label', )
#full_df = data_df.withColumn('id',monotonically_increasing_id())


# get the raw columns
raw_cols = data_df.columns
#raw_cols = lines.columns


data_df.printSchema()
#lines.printSchema()
data_df.na.drop(how="any").show(truncate=False)

root
 |-- value: string (nullable = true)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                      |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
'''
Ignore this
file_location = "immune.txt"
file_type = "txt"

infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

spark = SparkSession \
    .builder \
    .appName("Python Spark Sentiment Analysis example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.read.format(file_type).\
option("inferSchema", infer_schema).\
option("header", first_row_is_header).\
option("sep", delimiter).\
load(file_location)

df.show()
 
#temp_table_name = "immune.txt"
#df.createOrReplaceTempView(temp_table_name)
# Query the created temp table in a SQL cell *
#select * from `immune_txt`
# permanent_table_name = "immune_txt"
# df.write.format("parquet").saveAsTable(permanent_table_name)
'''

Py4JJavaError: ignored

In [ ]:
'''
Ignore this
data_df1 = sqlContext.createDataFrame(parts_rdd, ["text", "id", "label"])
data_df1.show(4)
'''

IndexError: ignored

In [ ]:
'''
Ignore this
from pyspark.sql.functions import lit
label_df = data_df.withColumn("label", lit(1.0)).show(4)
'''

In [ ]:
'''
Ignore this
from pyspark.sql.functions import monotonically_increasing_id
label_df = label_df.withColumn("id", monotonically_increasing_id())
label_df.show(4)
'''          

NameError: ignored

In [ ]:
'''
Ignore this
lang_df = data_df.withColumn("lang", check_lang_udf(data_df["value"]))
en_df = lang_df.filter(lang_df["lang"] == "en")
en_df.show(20)
'''

+-----+----+
|value|lang|
+-----+----+
|     |  en|
|     |  en|
|     |  en|
|     |  en|
|    8|  en|
|     |  en|
|    9|  en|
|     |  en|
|   10|  en|
|     |  en|
|     |  en|
|   11|  en|
|     |  en|
|     |  en|
|   12|  en|
|     |  en|
|     |  en|
|   13|  en|
|     |  en|
|     |  en|
+-----+----+
only showing top 20 rows



In [ ]:
'''
Ignore this
df_final = df_final.withColumn("monotonically_increasing_id", F.monotonically_increasing_id())
''' 

In [ ]:
'''
Ignore this
from pyspark.sql.functions import udf

@udf("double")
def const_col():
    return 1.0

label_df = data_df.withColumn('label', const_col())
label_df.show()
'''

+--------------------+-----+
|               value|label|
+--------------------+-----+
|[medRxiv preprint...|  1.0|
|                  []|  1.0|
|[1 Differential P...|  1.0|
|                  []|  1.0|
|[2 Geographic Ori...|  1.0|
|                  []|  1.0|
|[3 4 Owen R. Bake...|  1.0|
|                  []|  1.0|
|                 [8]|  1.0|
|                  []|  1.0|
|                 [9]|  1.0|
|                  []|  1.0|
|                [10]|  1.0|
|                  []|  1.0|
|[1) Division of I...|  1.0|
|                  []|  1.0|
|                [11]|  1.0|
|                  []|  1.0|
|[National Institu...|  1.0|
|                  []|  1.0|
+--------------------+-----+
only showing top 20 rows



In [ ]:
'''
Ignore this
import pyspark.sql.functions as F
id_df=label_df.withColumn('id', F.col('id')-1)
id_df.show()
'''

AnalysisException: ignored

In [8]:
# Remove stopwords from values
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords.words('english')
rm_stops_df = data_df.select(raw_cols).withColumn("stop_text", remove_stops_udf(data_df["value"]))
rm_stops_df.show(20)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
+--------------------+--------------------+
|               value|           stop_text|
+--------------------+--------------------+
| [Journal Pre-proof]| [Journal Pre-proof]|
|                  []|                  []|
|[Sleep disturbanc...|[Sleep disturbanc...|
|                  []|                  []|
|[Yanan liu , Xi W...|[Yanan liu , Xi W...|
|                  []|                  []|
|[PII: DOI: Refere...|[PII: DOI: Refere...|
|                  []|                  []|
|[S0165-0327(21)01...|[S0165-0327(21)01...|
|                  []|                  []|
|     [To appear in:]|     [To appear in:]|
|                  []|                  []|
|[Journal of Affec...|[Journal Affectiv...|
|                  []|                  []|
|[Received date: R...|[Received date: R...|
|                  []|                  []|
|[27 February 2021...|[27 February 2021...|
|

In [9]:
# Remove features from values
import re
import string
rm_features_df = rm_stops_df.select(raw_cols+["stop_text"]).withColumn("feat_text", remove_features_udf(rm_stops_df["stop_text"]))
rm_features_df.show(20)

+--------------------+--------------------+--------------------+
|               value|           stop_text|           feat_text|
+--------------------+--------------------+--------------------+
| [Journal Pre-proof]| [Journal Pre-proof]|   journal pre proof|
|                  []|                  []|                    |
|[Sleep disturbanc...|[Sleep disturbanc...|sleep disturbance...|
|                  []|                  []|                    |
|[Yanan liu , Xi W...|[Yanan liu , Xi W...|yanan liu  wang p...|
|                  []|                  []|                    |
|[PII: DOI: Refere...|[PII: DOI: Refere...|   pii doi reference|
|                  []|                  []|                    |
|[S0165-0327(21)01...|[S0165-0327(21)01...|             jad jad|
|                  []|                  []|                    |
|     [To appear in:]|     [To appear in:]|             appear |
|                  []|                  []|                    |
|[Journal of Affec...|[Jo

In [10]:
# Tagging text from value
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
tagged_df = rm_features_df.select(raw_cols+["feat_text"]).withColumn("tagged_text", tag_and_remove_udf(rm_features_df.feat_text))
tagged_df.show(20)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
+--------------------+--------------------+--------------------+
|               value|           feat_text|         tagged_text|
+--------------------+--------------------+--------------------+
| [Journal Pre-proof]|   journal pre proof|  journal pre proof |
|                  []|                    |                    |
|[Sleep disturbanc...|sleep disturbance...| sleep disturbanc...|
|                  []|                    |                    |
|[Yanan liu , Xi W...|yanan liu  wang p...| yanan liu wang p...|
|                  []|                    |                    |
|[PII: DOI: Refere...|   pii doi reference|  pii doi reference |
|                  []|                    |                    |
|[S0165-0327(21)01...|             jad jad|            jad jad |
|                  []|          

In [11]:
# Lemmatization for values
import re
import string
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
lemm_df = tagged_df.select(raw_cols+["tagged_text"]).withColumn("lemm_text", lemmatize_udf(tagged_df["tagged_text"]))
lemm_df.show(20)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
+--------------------+--------------------+--------------------+
|               value|         tagged_text|           lemm_text|
+--------------------+--------------------+--------------------+
| [Journal Pre-proof]|  journal pre proof |   journal pre proof|
|                  []|                    |                    |
|[Sleep disturbanc...| sleep disturbanc...|sleep disturbance...|
|                  []|                    |                    |
|[Yanan liu , Xi W...| yanan liu wang p...|yanan liu wang pe...|
|                  []|                    |                    |
|[PII: DOI: Refere...|  pii doi reference |   pii doi reference|
|                  []|                    |                    |
|[S0165-0327(21)01...|            jad jad |             jad jad|
|                  []|                    |                    |
|     [To appear in:]|             appea

In [12]:
check_blanks_df = lemm_df.select(raw_cols+["lemm_text"]).withColumn("is_blank", check_blanks_udf(lemm_df["lemm_text"]))
# remove blanks from lemmatized text
no_blanks_df = check_blanks_df.filter(check_blanks_df["is_blank"] == "False")

# drop duplicates from lemmatized text
dedup_df = no_blanks_df.dropDuplicates(['lemm_text'])

dedup_df.show(20)

+--------------------+--------------------+--------+
|               value|           lemm_text|is_blank|
+--------------------+--------------------+--------+
|[for age and gend...|age gender create...|   False|
|[Anaphylaxis, non...|anaphylaxis non a...|   False|
|          [Barriers]|             barrier|   False|
|[conditions, infe...|condition infecti...|   False|
|[disagreement amo...|disagreement inve...|   False|
|[186 liver diseas...|  disease adjustment|   False|
|[Figure 2: Cases-...|figure case infec...|   False|
|[29. Foley DA, Ph...|foley phuong pepl...|   False|
|[Some GPs mention...|gps mention prefe...|   False|
|[25 Israel A, She...|israel shenhar gr...|   False|
|[ Joining ambulan...|join ambulance da...|   False|
|[1 Khoury DS, Cro...|khoury cromer rey...|   False|
|[192 nine negativ...|negative sample f...|   False|
|[Number of partic...|number participan...|   False|
|[N =428,494 parti...|participant ethni...|   False|
|        [3 Priority]|            priority|   

In [13]:
# Rename final column as text for processing
final_df = dedup_df.selectExpr("lemm_text as text")
final_df.show()
final_df.printSchema()

+--------------------+
|                text|
+--------------------+
|join ambulance da...|
|  disease adjustment|
|reference healthy...|
|table report asso...|
|figure case infec...|
|negative sample f...|
| strength limitation|
|khoury cromer rey...|
|israel shenhar gr...|
|condition infecti...|
|anaphylaxis non a...|
|world health orga...|
|age gender create...|
|    publish november|
|               sandb|
|number participan...|
|gps mention prefe...|
|foley phuong pepl...|
|             barrier|
|            priority|
+--------------------+
only showing top 20 rows

root
 |-- text: string (nullable = true)



In [14]:
from pyspark.sql.functions import monotonically_increasing_id
# Create Unique ID for text
uid_df = final_df.withColumn("uid", monotonically_increasing_id())
uid_df.show(4)

+--------------------+---+
|                text|uid|
+--------------------+---+
|join ambulance da...|  0|
|  disease adjustment|  1|
|reference healthy...|  2|
|table report asso...|  3|
+--------------------+---+
only showing top 4 rows



In [15]:
# Create label with 1.0 being the constant for ML classifier
from pyspark.sql.functions import udf

@udf("double")
def const_col():
    return 1.0

label_df = uid_df.withColumn('label', const_col())
label_df.show()

+--------------------+---+-----+
|                text|uid|label|
+--------------------+---+-----+
|join ambulance da...|  0|  1.0|
|  disease adjustment|  1|  1.0|
|reference healthy...|  2|  1.0|
|table report asso...|  3|  1.0|
|figure case infec...|  4|  1.0|
|negative sample f...|  5|  1.0|
| strength limitation|  6|  1.0|
|khoury cromer rey...|  7|  1.0|
|israel shenhar gr...|  8|  1.0|
|condition infecti...|  9|  1.0|
|anaphylaxis non a...| 10|  1.0|
|world health orga...| 11|  1.0|
|age gender create...| 12|  1.0|
|    publish november| 13|  1.0|
|               sandb| 14|  1.0|
|number participan...| 15|  1.0|
|gps mention prefe...| 16|  1.0|
|foley phuong pepl...| 17|  1.0|
|             barrier| 18|  1.0|
|            priority| 19|  1.0|
+--------------------+---+-----+
only showing top 20 rows



In [16]:
data = label_df.select('uid','text','label')
data.show(20)

+---+--------------------+-----+
|uid|                text|label|
+---+--------------------+-----+
|  0|join ambulance da...|  1.0|
|  1|  disease adjustment|  1.0|
|  2|reference healthy...|  1.0|
|  3|table report asso...|  1.0|
|  4|figure case infec...|  1.0|
|  5|negative sample f...|  1.0|
|  6| strength limitation|  1.0|
|  7|khoury cromer rey...|  1.0|
|  8|israel shenhar gr...|  1.0|
|  9|condition infecti...|  1.0|
| 10|anaphylaxis non a...|  1.0|
| 11|world health orga...|  1.0|
| 12|age gender create...|  1.0|
| 13|    publish november|  1.0|
| 14|               sandb|  1.0|
| 15|number participan...|  1.0|
| 16|gps mention prefe...|  1.0|
| 17|foley phuong pepl...|  1.0|
| 18|             barrier|  1.0|
| 19|            priority|  1.0|
+---+--------------------+-----+
only showing top 20 rows



In [17]:
# Split the data into training and test sets (40% held out for testing)
(trainingData, testData) = data.randomSplit([0.6, 0.4])

In [33]:
# Running Naive Bayes classifier.
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and nb.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
#data = tokenizer.transform(data)
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="rawFeatures")
#labelEncoder = StringIndexer(inputCol="text", outputCol='label')
#vectorizer = CountVectorizer(inputCol= "words", outputCol="rawFeatures")
idf = IDF(minDocFreq=2000, inputCol="rawFeatures", outputCol="features")
#assembler = VectorAssembler(inputCols=["hour", "mobile", "userFeatures"],outputCol="features")
#idfModel = idf.fit(data)

#lda = LDA(k=20, seed=1, optimizer="em")

# DecisionTree Classifier
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Linear Support Vector Classifer
#lsvc = LinearSVC((maxIter=10, regParam=0.1)

# Naive Bayes model
#nb = NaiveBayes(smoothing=2.0)

# Random Forest Classifier
#rfc = RandomForestClassifier()

# Pipeline Architecture for NB
#pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, nb])

# Pipeline Architecture for RFC
#pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, rfc])

# Pipeline Architecture for DT
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, dt])

# Pipeline Architecture for LSVC
#pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, lsvc])

# Train model.  This also runs the indexers.
#model = pipeline.fit(trainingData)
#model

In [34]:
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("text", "label", "prediction").show(5,True)

+--------------------+-----+----------+
|                text|label|prediction|
+--------------------+-----+----------+
|  disease adjustment|  1.0|       0.0|
|reference healthy...|  1.0|       0.0|
|table report asso...|  1.0|       0.0|
|figure case infec...|  1.0|       0.0|
| strength limitation|  1.0|       0.0|
+--------------------+-----+----------+
only showing top 5 rows



In [35]:
from pyspark.ml.evaluation import RegressionEvaluator
r_evaluator = RegressionEvaluator(predictionCol="prediction")
r_evaluator.evaluate(predictions)

1.0

In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
m_evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
m_evaluator.evaluate(predictions)

0.0

In [32]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
b_evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
b_evaluator.evaluate(predictions)

1.0

In [ ]:
'''
Ignore this
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Create ParamGrid and Evaluator for Cross Validation
paramGrid = ParamGridBuilder().addGrid(nb.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.5, 2.0]).build()
cvEvaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")

# Run Cross-validation
cv = CrossValidator(estimator=nb, estimatorParamMaps=paramGrid, evaluator=cvEvaluator)
cvModel = cv.fit(trainingData)

# Make predictions on testData. cvModel uses the bestModel.
cvPredictions = cvModel.transform(test)

# Evaluate bestModel found from Cross Validation
evaluator.evaluate(cvPredictions)
'''

IllegalArgumentException: ignored

In [24]:
from pyspark.sql.types import FloatType

from textblob import TextBlob

def sentiment_analysis(text):
    return TextBlob(text).sentiment.polarity

sentiment_analysis_udf = udf(sentiment_analysis , FloatType())

In [25]:
df  = label_df.withColumn("sentiment_score", sentiment_analysis_udf( label_df['text'] ))
df.show(20,True)

+--------------------+---+-----+---------------+
|                text|uid|label|sentiment_score|
+--------------------+---+-----+---------------+
|join ambulance da...|  0|  1.0|          0.025|
|  disease adjustment|  1|  1.0|            0.0|
|reference healthy...|  2|  1.0|            0.5|
|table report asso...|  3|  1.0|            0.0|
|figure case infec...|  4|  1.0|          0.125|
|negative sample f...|  5|  1.0|       -0.30625|
| strength limitation|  6|  1.0|            0.0|
|khoury cromer rey...|  7|  1.0|            0.0|
|israel shenhar gr...|  8|  1.0|    0.007142857|
|condition infecti...|  9|  1.0|            0.0|
|anaphylaxis non a...| 10|  1.0|            0.0|
|world health orga...| 11|  1.0|            0.4|
|age gender create...| 12|  1.0|          0.375|
|    publish november| 13|  1.0|            0.0|
|               sandb| 14|  1.0|            0.0|
|number participan...| 15|  1.0|            0.0|
|gps mention prefe...| 16|  1.0|   -0.011363637|
|foley phuong pepl..

In [37]:
def condition(r):
    if (r >=0.1):
        label = "positive"
    elif(r <= -0.1):
        label = "negative"
    else:
        label = "neutral"
    return label

sentiment_udf = udf(lambda x: condition(x), StringType())

In [27]:
'''
Take too long to process
# Pipeline for LDA model
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier
from pyspark.ml.clustering import LDA
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.feature import CountVectorizer

# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and nb.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
#data = tokenizer.transform(data)
vectorizer = CountVectorizer(inputCol= "words", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")
#idfModel = idf.fit(data)

lda = LDA(k=20, seed=1, optimizer="em")
lda
pipeline = Pipeline(stages=[tokenizer, vectorizer,idf, lda])


model = pipeline.fit(data)
'''

KeyboardInterrupt: ignored

In [ ]:
'''
Please Ignore
from pyspark.sql.types import ArrayType, StringType

def termsIdx2Term(vocabulary):
    def termsIdx2Term(termIndices):
        return [vocabulary[int(index)] for index in termIndices]
    return udf(termsIdx2Term, ArrayType(StringType()))

vectorizerModel = model.stages[1]
vocabList = vectorizerModel.vocabulary
final = ldatopics.withColumn("Terms", termsIdx2Term(vocabList)("termIndices"))
'''